In [7]:
import threading
import time

In [8]:
def threading_test(name):
    print(f'Background process started with {name}')
    time.sleep(3)
    print(f'Background thread finished')



Main thread finishes before the background thread is complete

In [9]:
if __name__ == '__main__':

    print(f'Am the main thread')
    t = threading.Thread(target = threading_test, args = ["Kapoorlabs"])
    t.start()
    # Main thread is not waiting for background thread to finish
    
    print(f'Main ended')

Am the main thread
Background process started with Kapoorlabs
Main ended


Add daemon to kill the background thread

In [10]:
if __name__ == '__main__':

    print(f'Am the main thread')
    t = threading.Thread(target = threading_test, args = ["Kapoorlabs"], daemon = True)
    t.start()
    # Main thread is not waiting for background thread to finish
    
    print(f'Main ended')

Am the main thread
Background process started with Kapoorlabs
Main ended


Use join() to wait for the background thread to finish execution

In [12]:
if __name__ == '__main__':

    print(f'Am the main thread')
    t = threading.Thread(target = threading_test, args = ["Kapoorlabs"])
    t.start()
    #Wait for background thread to finish
    t.join()
    print(f'Main ended')


Am the main thread
Background process started with Kapoorlabs
Background thread finished
Main ended


In [17]:
import concurrent.futures
import os

In [19]:
if __name__ == '__main__':
    nthreads = os.cpu_count()
    print(f'Available number of cpu {nthreads}')
    with concurrent.futures.ThreadPoolExecutor(max_workers = nthreads) as executor:
        executor.map(threading_test, ['Kapoorlabs', 'Paris', 'Berlin', 'India'])

Available number of cpu 16
Background process started with Kapoorlabs
Background process started with Paris
Background process started with Berlin
Background process started with India
Background thread finished
Background thread finished
Background thread finished
Background thread finished
